In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.ensemble import RandomForestClassifier
import time
import xgboost as xgb
import lightgbm as lgb
import warnings
import pickle

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plt.style.use(style='seaborn')
%matplotlib inline

In [2]:
%%time
target = pd.read_csv('target.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Wall time: 34 s


In [3]:
print(train.columns.tolist())

['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D4', 'D10', 'D15', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V1

In [4]:
test_id = pd.read_csv('test2.csv')
test_id = test_id['TransactionID']

In [5]:
imp_train = train[['TransactionAmt','card2','addr1','card1']]

In [6]:
%%time
# XGBOOST
xgmodel = xgb.XGBClassifier(
    n_estimators=300,
    bagging_fraction = 0.8993155305338455,
    colsample_bytree = 0.7463058454739352,
    feature_fraction = 0.7989765808988153,
    gamma = 0.6665437467229817,
    learning_rate = 0.013887824598276186,
    max_depth = 16,
    min_child_samples = 170,
    num_leaves = 220,
    reg_alpha = 0.39871702770778467,
    reg_lambda = 0.24309304355829786,
    subsample = 0.7
)

xgmodel.fit(imp_train, target)

[02:22:51] WARNING: ..\src\learner.cc:541: 
Parameters: { bagging_fraction, feature_fraction, min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:22:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 10min 37s


XGBClassifier(bagging_fraction=0.8993155305338455, base_score=0.5,
              booster='gbtree', colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7463058454739352,
              feature_fraction=0.7989765808988153, gamma=0.6665437467229817,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.013887824598276186, max_delta_step=0,
              max_depth=16, min_child_samples=170, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=300,
              n_jobs=4, num_leaves=220, num_parallel_tree=1, random_state=0,
              reg_alpha=0.39871702770778467, reg_lambda=0.24309304355829786,
              scale_pos_weight=1, subsample=0.7, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [9]:
imp_test = test[['TransactionAmt','card2','addr1','card1']]

In [10]:
y_pred_xg = xgmodel.predict_proba(imp_test)
sub = pd.DataFrame()
sub['TransactionID'] = test_id
sub['isFraud'] = y_pred_xg[:, 1]
sub.to_csv('55submission_xgb.csv', index=False)